<img src="https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/media/logo/newebac_logo_black_half.png" alt="ebac-logo">

---

# **Módulo** | Análise de Dados: COVID-19 Dashboard
Caderno de **Exercícios**<br> 
Professor [André Perez](https://www.linkedin.com/in/andremarcosperez/)

---

# **Tópicos**

<ol type="1">
  <li>Introdução;</li>
  <li>Análise Exploratória de Dados;</li>
  <li>Visualização Interativa de Dados;</li>
  <li>Storytelling.</li>
</ol>


---

# **Exercícios**

Este *notebook* deve servir como um guia para **você continuar** a construção da sua própria análise exploratória de dados interativa. Fique a vontate para copiar os códigos da aula mas busque explorar os dados ao máximo. Por fim, publique seu *notebook* no [Kaggle](https://www.kaggle.com/) e seu *dashboard* [Google Data Studio](https://datastudio.google.com/).

---

# **COVID Dashboard**

## 1\. Contexto

Em uma emissora de televisão, a editora-chefe entra em contato com o time de analistas de dados com um pedido urgente: uma apresentação detalhada sobre a COVID-19 para ser exibida durante a edição noturna do jornal. A principal preocupação é a praticidade e clareza dos dados, garantindo que sejam compreensíveis para todos os telespectadores, independentemente de seu nível de conhecimento técnico. Estamos em fevereiro de 2021.

Pontos Reiterados:

**Praticidade e Clareza:** As informações devem ser apresentadas de forma simples e direta, facilitando a compreensão.

**Usabilidade:** A interface deve ser intuitiva, permitindo que o apresentador, mesmo sem habilidades técnicas avançadas, consiga navegar e destacar informações com facilidade.

**Tendências e Velocidade:** A apresentação deve focar nas tendências atuais e na velocidade de propagação do vírus, ajudando a informar a população sobre a situação em tempo real.

**Localização:** Como o jornal é transmitido no estado de Minas Gerais, é crucial incluir informações específicas da região para manter o interesse e relevância para os telespectadores locais.

**Minimizar Alarme:** Para evitar pânico, os números de mortes e casos confirmados serão exibidos em tamanhos menores, reduzindo o impacto visual e emocional dessas informações.

Após a reunião, a equipe de analistas de dados iniciou a criação de uma Dashboard que fosse simples, clara e eficiente, pronta para ser apresentada no jornal.

 - **Dashboard**: 
  - [Google Data Studio](`https://lookerstudio.google.com/reporting/9e522d95-45a0-476e-83b4-4f71198b0d50`).
 - **Fontes**: 
  - Casos pela universidade John Hopkins ([link](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports));
  - Vacinação pela universidade de Oxford ([link](https://covid.ourworldindata.org/data/owid-covid-data.csv)).

## 2\. Pacotes e bibliotecas

In [69]:
import math
from typing import Iterator
from datetime import datetime, timedelta

import numpy as np
import pandas as pd

## 3\. Extração

### Extração do primeiro CSV

In [70]:
 # Baixar arquivo
cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-12-2021.csv', sep=',')

In [71]:
# Função para definir formato da data
def date_range(start_date: datetime, end_date: datetime) -> Iterator[datetime]:
  date_range_days: int = (end_date - start_date).days
  for lag in range(date_range_days):
    yield start_date + timedelta(lag)

start_date = datetime(2021,  1,  1)
end_date   = datetime(2021, 12, 31)

In [73]:
# Função para gerar um intervalo de datas
def date_range(start_date, end_date):
    for n in range(int((end_date - start_date).days) + 1):
        yield start_date + timedelta(n)

cases_list = []
start_date = datetime.strptime("2021-01-01", '%Y-%m-%d')
end_date = datetime.strptime("2021-12-31", '%Y-%m-%d')

for date in date_range(start_date=start_date, end_date=end_date):
    date_str = date.strftime('%m-%d-%Y')
    data_source_url = f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{date_str}.csv'
    
    case = pd.read_csv(data_source_url, sep=',')
    case = case.drop(['FIPS', 'Admin2', 'Last_Update', 'Lat', 'Long_', 'Recovered', 'Active', 'Combined_Key', 'Case_Fatality_Ratio'], axis=1)
    case = case.query('Country_Region == "Brazil"').reset_index(drop=True)
    case['Date'] = pd.to_datetime(date.strftime('%Y-%m-%d'))
    
    cases_list.append(case)

# Concatenar todos os DataFrames em um único DataFrame
cases = pd.concat(cases_list, ignore_index=True)

# Verificar as primeiras linhas do DataFrame resultante
print(cases.head())


  Province_State Country_Region  Confirmed  Deaths  Incident_Rate       Date
0           Acre         Brazil      41689     796    4726.992352 2021-01-01
1        Alagoas         Brazil     105091    2496    3148.928928 2021-01-01
2          Amapa         Brazil      68361     926    8083.066602 2021-01-01
3       Amazonas         Brazil     201574    5295    4863.536793 2021-01-01
4          Bahia         Brazil     494684    9159    3326.039611 2021-01-01


Pensando no contexto proposto, fiz primeiro um arquivo com o end_date final em janeiro, que usei para montar a Dash. Depois montei o arquivo contendo todos os dados de 2021 para subir no Git.

In [74]:
# Renomear colunas
cases = cases.rename(
  columns={
    'Province_State': 'state',
    'Country_Region': 'country'
  }
)

for col in cases.columns:
  cases = cases.rename(columns={col: col.lower()})

In [75]:
# Renomear estados
states_map = {
    'Amapa': 'Amapá',
    'Ceara': 'Ceará',
    'Espirito Santo': 'Espírito Santo',
    'Goias': 'Goiás',
    'Para': 'Pará',
    'Paraiba': 'Paraíba',
    'Parana': 'Paraná',
    'Piaui': 'Piauí',
    'Rondonia': 'Rondônia',
    'Sao Paulo': 'São Paulo'
}

cases['state'] = cases['state'].apply(lambda state: states_map.get(state) if state in states_map.keys() else state)

In [76]:
# Função para criar colunas específicas
cases['month'] = cases['date'].apply(lambda date: date.strftime('%Y-%m'))
cases['year']  = cases['date'].apply(lambda date: date.strftime('%Y'))

cases['population'] = round(100000 * (cases['confirmed'] / cases['incident_rate']))
cases = cases.drop('incident_rate', axis=1)

In [77]:
# Função para criar coluna e calcular média
def get_trend(rate: float) -> str:
    if np.isnan(rate):
        return np.NaN

    if rate < 0.75:
        status = 'downward'
    elif rate > 1.15:
        status = 'upward'
    else:
        status = 'stable'

    return status

cases_list = []

for state in cases['state'].drop_duplicates():
    cases_per_state = cases.query(f'state == "{state}"').reset_index(drop=True)
    cases_per_state = cases_per_state.sort_values(by=['date'])

    # Calcular média móvel de 7 dias para casos confirmados
    cases_per_state['confirmed_1d'] = cases_per_state['confirmed'].diff(periods=1)
    cases_per_state['confirmed_moving_avg_7d'] = np.ceil(cases_per_state['confirmed_1d'].rolling(window=7).mean())

    # Calcular a métrica de tendência
    cases_per_state['confirmed_moving_avg_7d_rate_14d'] = cases_per_state['confirmed_moving_avg_7d'] / cases_per_state['confirmed_moving_avg_7d'].shift(periods=14)
    cases_per_state['confirmed_trend'] = cases_per_state['confirmed_moving_avg_7d_rate_14d'].apply(get_trend)

    # Calcular média móvel de 7 dias para mortes
    cases_per_state['deaths_1d'] = cases_per_state['deaths'].diff(periods=1)
    cases_per_state['deaths_moving_avg_7d'] = np.ceil(cases_per_state['deaths_1d'].rolling(window=7).mean())

    # Calcular a métrica de tendência
    cases_per_state['deaths_moving_avg_7d_rate_14d'] = cases_per_state['deaths_moving_avg_7d'] / cases_per_state['deaths_moving_avg_7d'].shift(periods=14)
    cases_per_state['deaths_trend'] = cases_per_state['deaths_moving_avg_7d_rate_14d'].apply(get_trend)

    # Adicionar o DataFrame processado à lista
    cases_list.append(cases_per_state)

# Concatenar todos os DataFrames em um único DataFrame
cases = pd.concat(cases_list, ignore_index=True)

cases.head()

,state,country,confirmed,deaths,date,month,year,population,confirmed_1d,confirmed_moving_avg_7d,confirmed_moving_avg_7d_rate_14d,confirmed_trend,deaths_1d,deaths_moving_avg_7d,deaths_moving_avg_7d_rate_14d,deaths_trend
0,Acre,Brazil,41689,796,2021-01-01,2021-01,2021,881935.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Acre,Brazil,41941,798,2021-01-02,2021-01,2021,881935.0,252.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN
2,Acre,Brazil,42046,802,2021-01-03,2021-01,2021,881935.0,105.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN
3,Acre,Brazil,42117,806,2021-01-04,2021-01,2021,881935.0,71.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN
4,Acre,Brazil,42170,808,2021-01-05,2021-01,2021,881935.0,53.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN


In [78]:
# Ordenar tipos das colunas
cases['population'] = cases['population'].astype('Int64')
cases['confirmed_1d'] = cases['confirmed_1d'].astype('Int64')
cases['confirmed_moving_avg_7d'] = cases['confirmed_moving_avg_7d'].astype('Int64')
cases['deaths_1d'] = cases['deaths_1d'].astype('Int64')
cases['deaths_moving_avg_7d'] = cases['deaths_moving_avg_7d'].astype('Int64')

In [79]:
# Ajustar ordem das colunas
cases = cases[['date', 'country', 'state', 'population', 'confirmed', 'confirmed_1d', 'confirmed_moving_avg_7d', 'confirmed_moving_avg_7d_rate_14d', 'confirmed_trend', 'deaths', 'deaths_1d', 'deaths_moving_avg_7d', 'deaths_moving_avg_7d_rate_14d', 'deaths_trend', 'month', 'year']]

In [80]:
cases.tail()

,date,country,state,population,confirmed,confirmed_1d,confirmed_moving_avg_7d,confirmed_moving_avg_7d_rate_14d,confirmed_trend,deaths,deaths_1d,deaths_moving_avg_7d,deaths_moving_avg_7d_rate_14d,deaths_trend,month,year
9850,2021-12-27,Brazil,Tocantins,1572866,234113,0,0,0.000000,downward,3927,0,0,0.0,downward,2021-12,2021
9851,2021-12-28,Brazil,Tocantins,1572866,234964,851,122,2.837209,upward,3933,6,1,1.0,stable,2021-12,2021
9852,2021-12-29,Brazil,Tocantins,1572866,235340,376,176,inf,upward,3936,3,2,inf,upward,2021-12,2021
9853,2021-12-30,Brazil,Tocantins,1572866,235558,218,207,inf,upward,3939,3,2,inf,upward,2021-12,2021
9854,2021-12-31,Brazil,Tocantins,1572866,235558,0,207,inf,upward,3939,0,2,inf,upward,2021-12,2021


In [81]:
# Criar csv
cases.to_csv('./covid-cases.csv', sep=',', index=False)

### Extração do Segundo CSV

In [46]:
# Baixar dados
vaccines = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv', sep=',', parse_dates=[3], infer_datetime_format=True)

/tmp/ipykernel_10223/738699849.py:1: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  vaccines = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv', sep=',', parse_dates=[3], infer_datetime_format=True)


In [47]:
# Organizando colunas
vaccines = vaccines.query('location == "Brazil"').reset_index(drop=True)
vaccines = vaccines[['location', 'population', 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'total_boosters', 'date']]

In [48]:
# Preenchendo dados faltantes
vaccines = vaccines.fillna(method='ffill')

/tmp/ipykernel_10223/1781271583.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  vaccines = vaccines.fillna(method='ffill')


In [49]:
# Ajustando índice com a data
vaccines = vaccines[(vaccines['date'] >= '2021-01-01') & (vaccines['date'] <= '2021-12-31')].reset_index(drop=True)

In [50]:
# Renomeando colunas
vaccines = vaccines.rename(
  columns={
    'location': 'country',
    'total_vaccinations': 'total',
    'people_vaccinated': 'one_shot',
    'people_fully_vaccinated': 'two_shots',
    'total_boosters': 'three_shots',
  }
)

In [51]:
# Funções para criar colunas - data - porcentagem
vaccines['month'] = vaccines['date'].apply(lambda date: date.strftime('%Y-%m'))
vaccines['year']  = vaccines['date'].apply(lambda date: date.strftime('%Y'))

vaccines['one_shot_perc'] = round(vaccines['one_shot'] / vaccines['population'], 4)
vaccines['two_shots_perc'] = round(vaccines['two_shots'] / vaccines['population'], 4)
vaccines['three_shots_perc'] = round(vaccines['three_shots'] / vaccines['population'], 4)

vaccines['population'] = vaccines['population'].astype('Int64')
vaccines['total'] = vaccines['total'].astype('Int64')
vaccines['one_shot'] = vaccines['one_shot'].astype('Int64')
vaccines['two_shots'] = vaccines['two_shots'].astype('Int64')
vaccines['three_shots'] = vaccines['three_shots'].astype('Int64')

# Organizando colunas
vaccines = vaccines[['date', 'country', 'population', 'total', 'one_shot', 'one_shot_perc', 'two_shots', 'two_shots_perc', 'three_shots', 'three_shots_perc', 'month', 'year']]

In [52]:
# Criando csv
vaccines.to_csv('./covid-vaccines.csv', sep=',', index=False)